In [1]:
# data wrangling
import pandas as pd

# plotting
import matplotlib.pyplot as plt

# other
import time

# sklearn
from sklearn.model_selection import train_test_split

# local files
from sofascores import compute_sofa

In [2]:
# load data
DATA_FILEPATH = "all_hourly_data.h5"
patients = pd.read_hdf(DATA_FILEPATH, "patients")
vitals_labs_mean = pd.read_hdf(DATA_FILEPATH, "vitals_labs_mean")
interventions = pd.read_hdf(DATA_FILEPATH, "interventions")

In [14]:
print(patients.shape)
print(patients.head())

(34472, 28)
                              gender              ethnicity        age  \
subject_id hadm_id icustay_id                                            
3          145834  211552          M                  WHITE  76.526792   
4          185777  294638          F                  WHITE  47.845047   
6          107064  228232          F                  WHITE  65.942297   
9          150750  220597          M  UNKNOWN/NOT SPECIFIED  41.790228   
11         194540  229441          F                  WHITE  50.148295   

                              insurance           admittime  \
subject_id hadm_id icustay_id                                 
3          145834  211552      Medicare 2101-10-20 19:08:00   
4          185777  294638       Private 2191-03-16 00:28:00   
6          107064  228232      Medicare 2175-05-30 07:15:00   
9          150750  220597      Medicaid 2149-11-09 13:06:00   
11         194540  229441       Private 2178-04-16 06:18:00   

                           

In [ ]:
print(patients.ethnicity.nunique())
for u in patients.ethnicity.unique():
    print(u)

In [18]:
print(patients.insurance.nunique())
for u in patients.insurance.unique():
    print(u)

5
Medicare
Private
Medicaid
Government
Self Pay


In [ ]:
print(patients.insurance.nunique())
for u in patients.insurance.unique():
    print(u)